In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

from ipywidgets import interact

import pandas as pd
import scipy.stats as stats
import pymc as pm
import seaborn as sns
import arviz as az

Populating the interactive namespace from numpy and matplotlib


# Question 1

During your internship at EPCOR, you are given a dataset containing 10,000 observations of monthly utility bills (in Canadian dollars) for Edmonton houses over the last couple of years. Along with the monthly bill, you are also given:

- `avg_temperature`: the average temperature during the billing month (in Celsius).
- `household_size`: the number of people living in the house during the billing month.
- `house_taxes`: the yearly property taxes according to the last tax notice (in Canadian dollars).

Under the consent of residents, the City of Edmonton kindly provided household and taxes information for the dataset. This information was matched, but it's not perfect.

Your goal is to determine whether the utility bill amount can be predicted using the other three variables.

## A (15 points)

Load the data from the file `EPCOR1.csv` (link provided below) into a Pandas DataFrame. Clean the data by removing any corrupted values.

Do you think using mean imputation is a better approach to deal with corrupted observations in this case? Explain.

In [30]:
path_to_data = 'https://raw.githubusercontent.com/ccontrer/MATH509-Winter2025-JupyterNotebooks/main/Data/Epcor1.csv'
data = pd.read_csv(path_to_data, index_col=0)
display(data.describe())

,household_size,house_taxes,bill
count,10000.000000,10000.000000,10000.000000
mean,2.098500,4107.637598,310.312478
std,1.175388,6314.119179,13.754624
min,0.000000,682.060000,287.571982
25%,1.000000,2718.060000,300.194427
50%,2.000000,3613.900000,307.332263
75%,3.000000,4519.920000,317.376281
max,8.000000,99999.000000,446.639790


In [31]:
data2 = data.loc[data['house_taxes'] == 99999]
display(data2.describe())
#removed taxes at 99999 and households at 0 as those are corrupted and cannot occur
data.loc[data['house_taxes'] == 99999, 'house_taxes'] = data.loc[data['house_taxes'] != 99999, 'house_taxes'].mean()
data.loc[data['household_size'] == 0, 'household_size'] = data.loc[data['household_size'] != 0, 'household_size'].mean()
display(data.describe())

#Overall probably the best method is mean imputation. looking at the corrupted data of taxes the household sizes are pretty random and bills are also random.
#using the mean value is fine for taxes. For household size there is no better option. We cannot tell what size would be better, so using mean imputation is the best option as well

,household_size,house_taxes,bill
count,41.000000,41.0,41.000000
mean,2.121951,99999.0,308.665960
std,1.122389,0.0,11.583652
min,0.000000,99999.0,291.158458
25%,1.000000,99999.0,300.016359
50%,2.000000,99999.0,304.934159
75%,3.000000,99999.0,315.883404
max,5.000000,99999.0,345.590065


<ipython-input-31-ea521e158cbd>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.2161791107825537' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[data['household_size'] == 0, 'household_size'] = data.loc[data['household_size'] != 0, 'household_size'].mean()


,household_size,house_taxes,bill
count,10000.000000,10000.000000,10000.000000
mean,2.216179,3712.864442,310.312478
std,1.065158,1417.384687,13.754624
min,1.000000,682.060000,287.571982
25%,1.000000,2718.060000,300.194427
50%,2.000000,3613.900000,307.332263
75%,3.000000,4499.560000,317.376281
max,8.000000,13641.200000,446.639790


## B (10 points)

Create scatter plots for the utility bill versus the average temperature and the property taxes in different subplots within the same figure.

## C (10 points)

Preprocess the data by standardizing all predictor variables.

## D (25 points)

Propose prior distributions for a simple linear model for `bill` using `avg_temperature` and `household_size` as predictors. Create prior predictive plots. Use MCMC sampling (make 4 chains with 1000 samples and 1000 warm-up steps) to estimate the posterior probability (it should take less than 20 seconds). Display a summary and plot all posterior distributions.

Justify your choice of prior distributions.

## E (10 points)

Based on the scatter plot obtained in part B, do you think this is a reasonable model? Interpret the mean of the posterior for each of the parameters. Comment on the relationship between the monthly average temperature and the utility bill amount. How would you approach modeling the utility bill as a function of the average monthly temperature?

## F (20 points)

Extend the linear model by adding `house_taxes` as a predictor. Use MCMC sampling (again, make 4 chains with 1000 samples and 1000 warm-up steps) to estimate the posterior probability (it should take less than 25 seconds). Display a summary and plot all posterior distributions.

Justify your choice of prior distribution.

## G (10 points)

Interpret the mean of the posterior for the new parameter. Compare the other mean posterior values with those obtained in part E. Based on the scatter plot you obtained in part B, do you think this is a reasonable model? Comment on the relationship between the yearly property taxes and the utility bill amount. What do you think is happening here?